In [1]:
!unzip /content/drive/MyDrive/yolov52.zip -d /content/output_folder

Archive:  /content/drive/MyDrive/yolov52.zip
   creating: /content/output_folder/content/yolov5/
  inflating: /content/output_folder/content/yolov5/README.zh-CN.md  
   creating: /content/output_folder/content/yolov5/.git/
   creating: /content/output_folder/content/yolov5/.git/hooks/
  inflating: /content/output_folder/content/yolov5/.git/hooks/update.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-rebase.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/prepare-commit-msg.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/applypatch-msg.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/post-update.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-commit.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-merge-commit.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/commit-msg.sample  
  inflating: /content/output_folder/content

In [4]:
import os
os.chdir('/content/output_folder/content/yolov5')

In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.4 MB/s eta 0:00:00


In [8]:
import torch
from models.experimental import attempt_load
from utils.torch_utils import fuse_conv_and_bn

# 1. Load your 90% mAP Nano model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights_path = '/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.pt'
model = attempt_load(weights_path, device=device)

# 2. Set to training mode (Fixes your AssertionError)
model.train()

# 3. Manual Fusion for YOLOv5
# This combines Conv + BN layers which is critical for INT8 performance
for m in model.modules():
    if type(m) is torch.nn.Sequential and len(m) == 2:
        if isinstance(m[0], torch.nn.Conv2d) and isinstance(m[1], torch.nn.BatchNorm2d):
            m[0] = fuse_conv_and_bn(m[0], m[1])
            m[1] = torch.nn.Identity()

# 4. Apply QAT Configuration using the reliable 'qnnpack' (standard for mobile/ARM)
# or 'fbgemm' (standard for x86/Server)
model.qconfig = torch.ao.quantization.get_default_qat_qconfig('qnnpack')

# 5. Prepare for QAT
torch.ao.quantization.prepare_qat(model, inplace=True)

print("✅ Success: QAT observers inserted and model set to training mode.")

Fusing layers... 
Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs
/tmp/ipython-input-1476176902.py:26: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  torch.ao.quantization.prepare_qat(model, inplace=True)


✅ Success: QAT observers inserted and model set to training mode.


In [10]:
import yaml

# Create a copy of the low-augmentation hyps for the final accuracy boost
with open('/content/output_folder/content/yolov5/data/hyps/hyp.scratch-low.yaml', 'r') as f:
    hyp = yaml.safe_load(f)

# Optimized for fine-tuning at high resolution (1024)
hyp['lr0'] = 0.001      # Lower starting rate for accuracy
hyp['lrf'] = 0.01       # Final LR factor
hyp['warmup_epochs'] = 3.0
hyp['mosaic'] = 0.0     # Disable mosaic for the final polish (cleaner grain edges)

with open('/content/output_folder/content/yolov5/data/hyps/hyp.rice_finetune.yaml', 'w') as f:
    yaml.dump(hyp, f)

In [12]:
# Run 10-20 epochs of fine-tuning at high resolution (1024)
# This recovers the accuracy usually lost in standard INT8 conversion
!python train.py --img 1024 \
                 --batch 16 \
                 --epochs 20 \
                 --data data.yaml \
                 --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.pt \
                 --name rice_nano_QAT_final \
                 --hyp /content/output_folder/content/yolov5/data/hyps/hyp.rice_finetune.yaml \
                 --exist-ok

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-12-23 18:22:36.291550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766514156.324111    6909 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766514156.334088    6909 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766514156.357589    6909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766514156.357632    6909 computation_placer.cc:177] computation placer already registere

In [13]:
# The --int8 flag here will use the quantization parameters learned in Step 5
!python export.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt \
                 --include onnx \
                 --int8 \
                 --data /content/output_folder/content/yolov5/data.yaml \
                 --img 1024 \
                 --simplify

export: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt'], imgsz=[1024], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=True, per_tensor=False, dynamic=False, cache=, simplify=True, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CPU

Fusing layers... 
Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs

PyTorch: starting from /content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt with output shape (1, 64512, 13) (4.1 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxscript'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 8 packages in 141ms
Prepared 3 packages in 1.2

In [15]:
!pip install onnx-simplifier -q
!python -m onnxsim rice_nano_final_int8.onnx rice_nano_final_int8.onnx -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 88.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 106.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 105.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 105.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 64.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

In [16]:
import numpy as np
from onnxruntime.quantization import CalibrationDataReader
import cv2
import os

class RiceDataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder, model_path):
        self.image_folder = calibration_image_folder
        self.image_list = [os.path.join(self.image_folder, f) for f in os.listdir(self.image_folder) if f.endswith(('.jpg', '.png'))][:20]
        self.preprocess_flag = True
        self.enum_data = None

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter([self.preprocess(img_path) for img_path in self.image_list])
        return next(self.enum_data, None)

    def preprocess(self, img_path):
        # Match your training resolution (1024)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (1024, 1024))
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1)) # HWC to CHW
        img = np.expand_dims(img, axis=0) # CHW to NCHW
        return {"images": img} # 'images' must match the input name of your ONNX model

In [17]:
from onnxruntime.quantization import quantize_static, QuantFormat, QuantType

input_model = "/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.onnx"
output_model = "/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/rice_nano_final_int8.onnx"
val_images_path = "/content/output_folder/content/yolov5/valid/images" # Point to your validation images

# Initialize data reader
dr = RiceDataReader(val_images_path, input_model)

# Execute Quantization
quantize_static(
    input_model,
    output_model,
    calibration_data_reader=dr,
    quant_format=QuantFormat.QOperator, # This is key for small size
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QInt8
)

import os
final_size = os.path.getsize(output_model) / (1024 * 1024)
print(f"✅ Final Compressed Model Size: {final_size:.2f} MB")

✅ Final Compressed Model Size: 2.04 MB


In [29]:
# Go back to the export script, but use the --grid flag and ensure --inplace is set
# Export to a clean FP32 ONNX with grid scaling included
!python export.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt \
                 --include onnx \
                 --img 1024 \
                 --data /content/output_folder/content/yolov5/data.yaml \
                 --inplace \
                 --simplify \
                 --opset 12

export: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt'], imgsz=[1024], batch_size=1, device=cpu, half=False, inplace=True, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=True, mlmodel=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CPU

Fusing layers... 
Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs

PyTorch: starting from /content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.pt with output shape (1, 64512, 13) (4.1 MB)

ONNX: starting export with onnx 1.20.0...
W1223 19:02:46.667000 17507 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_versi

In [32]:
import numpy as np
import cv2
import os
import onnxruntime as ort
from onnxruntime.quantization import CalibrationDataReader

class RiceDataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder, model_path):
        # 1. Verify the folder path
        if not os.path.exists(calibration_image_folder):
            raise FileNotFoundError(f"Folder not found: {calibration_image_folder}")

        # 2. Get image list (check for multiple extensions)
        self.image_list = [os.path.join(calibration_image_folder, f) for f in os.listdir(calibration_image_folder)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:30] # Use 30 images for better calibration

        if len(self.image_list) == 0:
            raise ValueError(f"No images found in {calibration_image_folder}. Check your path!")

        # 3. Automatically get the input name from the ONNX model
        session = ort.InferenceSession(model_path)
        self.input_name = session.get_inputs()[0].name
        print(f"✅ DataReader initialized with {len(self.image_list)} images.")
        print(f"✅ Model input name detected as: '{self.input_name}'")

        self.enum_data = iter(self.image_list)

    def get_next(self):
        img_path = next(self.enum_data, None)
        if img_path is None:
            return None

        # Preprocessing
        img = cv2.imread(img_path)
        img = cv2.resize(img, (1024, 1024))
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1)) # HWC to CHW
        img = np.expand_dims(img, axis=0) # CHW to NCHW

        return {self.input_name: img}

# --- RE-RUN QUANTIZATION ---
from onnxruntime.quantization import quantize_static, QuantFormat, QuantType

# Ensure these paths are 100% correct in your Colab
input_model = "/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/best.onnx"
output_model = "/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/rice_nano_final_1mb.onnx"
val_images = "/content/output_folder/content/yolov5/test/images" # Adjust if your test images are elsewhere

# Initialize the new reader
dr = RiceDataReader(val_images, input_model)

quantize_static(
    input_model,
    output_model,
    calibration_data_reader=dr,
    quant_format=QuantFormat.QOperator,
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QInt8
)

import os
print(f"✅ Final Success! Size: {os.path.getsize(output_model) / (1024*1024):.2f} MB")

✅ DataReader initialized with 15 images.
✅ Model input name detected as: 'images'


✅ Final Success! Size: 2.04 MB


In [33]:
import sys
import torch
import onnxruntime as ort
from tqdm import tqdm
# Ensure we can import from the yolov5 directory
sys.path.append('/content/output_folder/content/yolov5')
from utils.general import non_max_suppression, scale_boxes, xywh2xyxy, box_iou
from utils.metrics import ap_per_class

In [45]:
import os
import cv2
import yaml
import numpy as np
import torch
import onnxruntime as ort
from tqdm import tqdm
import sys

# Ensure YOLOv5 utils are accessible
sys.path.append('/content/output_folder/content/yolov5')
from utils.general import non_max_suppression, xywh2xyxy, box_iou
from utils.metrics import ap_per_class

def evaluate_onnx_accuracy(model_path, data_yaml, img_size=1024):
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    session = ort.InferenceSession(model_path, providers=providers)
    input_name = session.get_inputs()[0].name

    with open(data_yaml, 'r') as f:
        data = yaml.safe_load(f)

    test_path = '/content/output_folder/content/yolov5/test/images'
    image_files = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.lower().endswith(('.jpg', '.png'))]

    stats, seen = [], 0
    iouv = torch.linspace(0.5, 0.95, 10)

    print(f"🚀 Starting final evaluation on {len(image_files)} images...")

    for img_path in tqdm(image_files):
        img0 = cv2.imread(img_path)
        img = cv2.resize(img0, (img_size, img_size))
        img = img[:, :, ::-1].transpose((2, 0, 1))
        img = np.ascontiguousarray(img).astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=0)

        out = session.run(None, {input_name: img})[0]
        out = torch.from_numpy(out)

        # Scale coordinates and apply sigmoid if needed
        if out[..., :4].max() > 1.5: # 1.5 threshold to account for slight dequantization noise
            out[..., :4] /= img_size
        if out[..., 4:].max() > 1.0:
            out[..., 4:] = torch.sigmoid(out[..., 4:])

        preds = non_max_suppression(out, conf_thres=0.0001, iou_thres=0.45)

        # --- FIXED LABEL PATH LOGIC ---
        # 1. Change directory from /images/ to /labels/
        lbl_dir = os.path.dirname(img_path).replace('/images', '/labels')
        # 2. Change ONLY the final extension to .txt
        lbl_name = os.path.splitext(os.path.basename(img_path))[0] + '.txt'
        label_path = os.path.join(lbl_dir, lbl_name)
        # ------------------------------

        nl = 0
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path).reshape(-1, 5)
            nl = len(labels)

        if nl:
            tcls = labels[:, 0]
            tbox = xywh2xyxy(torch.from_numpy(labels[:, 1:]))
            targets = torch.cat((torch.from_numpy(tcls)[:, None], tbox), 1)
        else:
            targets = torch.zeros((0, 5))

        for i, det in enumerate(preds):
            seen += 1
            if len(det) == 0:
                if nl: stats.append((torch.zeros(0, 10, dtype=torch.bool), torch.Tensor(), torch.Tensor(), torch.from_numpy(tcls)))
                continue

            if nl:
                correct = torch.zeros(len(det), 10, dtype=torch.bool)
                iou = box_iou(targets[:, 1:], det[:, :4])
                for j, v in enumerate(iouv):
                    x = torch.where((iou >= v) & (targets[:, 0:1] == det[:, 5]))
                    if x[0].shape[0]:
                        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()
                        matches = matches[matches[:, 2].argsort()[::-1]]
                        matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                        matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
                        correct[matches[:, 1].astype(int), j] = True

            stats.append((correct.cpu(), det[:, 4].cpu(), det[:, 5].cpu(), torch.from_numpy(tcls).cpu()))

    stats = [torch.cat(x, 0).numpy() for x in zip(*stats)]
    if len(stats) and stats[0].any():
        tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, names=data['names'])
        print(f"\n✅ SUCCESS: mAP50 is {ap[:, 0].mean():.4f}")
        return ap[:, 0].mean()
    else:
        print("❌ Detection match failed. Try lowering 'conf_thres' to 0.0001 in non_max_suppression.")



# --- EXECUTE ---
model_to_test = "/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/rice_nano_final_1mb.onnx"
final_map = evaluate_onnx_accuracy(model_to_test, "/content/output_folder/content/yolov5/data.yaml")

🚀 Starting final evaluation on 15 images...


100%|██████████| 15/15 [00:02<00:00,  6.14it/s]

❌ Detection match failed. Try lowering 'conf_thres' to 0.0001 in non_max_suppression.


In [43]:
test_img = '/content/output_folder/content/yolov5/test/images/1700059259412_jpg.rf.86e160c4ccbaac8e6d4093ffb4cfecab.jpg' # Pick a real name
test_label = test_img.replace('/images/', '/labels/').replace('.jpg', '.txt')
print(f"Checking for label at: {test_label}")
print(f"Exists? {os.path.exists(test_label)}")

Checking for label at: /content/output_folder/content/yolov5/test/labels/1700059259412_jpg.rf.86e160c4ccbaac8e6d4093ffb4cfecab.txt
Exists? True


In [46]:
import os
import cv2
import yaml
import numpy as np
import torch
import onnxruntime as ort
from tqdm import tqdm
import sys

# Ensure YOLOv5 utils are accessible
sys.path.append('/content/output_folder/content/yolov5')
from utils.general import non_max_suppression, xywh2xyxy, box_iou
from utils.metrics import ap_per_class

def evaluate_onnx_accuracy(model_path, data_yaml, img_size=1024):
    # 1. Setup Session
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
    session = ort.InferenceSession(model_path, providers=providers)
    input_name = session.get_inputs()[0].name

    with open(data_yaml, 'r') as f:
        data = yaml.safe_load(f)

    # 2. Use the verified path from your previous step
    test_path = '/content/output_folder/content/yolov5/test/images'
    image_files = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.lower().endswith(('.jpg', '.png'))]

    stats, seen = [], 0
    iouv = torch.linspace(0.5, 0.95, 10)

    print(f"🚀 Running Robust Evaluation on {len(image_files)} images...")

    for img_path in tqdm(image_files):
        img0 = cv2.imread(img_path)
        img = cv2.resize(img0, (img_size, img_size))
        img = img[:, :, ::-1].transpose((2, 0, 1))  # BGR to RGB
        img = np.ascontiguousarray(img).astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=0)

        # Inference
        out = session.run(None, {input_name: img})[0]
        out = torch.from_numpy(out)

        # --- CRITICAL: MANUAL TENSOR REPAIR ---
        # If values are raw (like 36.0 or 4.6), we must normalize and sigmoid
        if out[..., :4].max() > 1.1:
            out[..., :4] /= img_size # Scale coordinates to 0-1 range

        # Apply Sigmoid to Objectness (col 4) and Class Scores (col 5+)
        # This is usually why mAP is 0.0 - the scores are raw logits
        out[..., 4:] = torch.sigmoid(out[..., 4:])
        # --------------------------------------

        # NMS with very low threshold to see if ANY detections exist
        preds = non_max_suppression(out, conf_thres=0.001, iou_thres=0.45)

        # Fix label path using Roboflow-safe logic
        lbl_dir = os.path.dirname(img_path).replace('/images', '/labels')
        lbl_name = os.path.splitext(os.path.basename(img_path))[0] + '.txt'
        label_path = os.path.join(lbl_dir, lbl_name)

        nl = 0
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path).reshape(-1, 5)
            nl = len(labels)

        if nl:
            tcls = labels[:, 0]
            tbox = xywh2xyxy(torch.from_numpy(labels[:, 1:]))
            targets = torch.cat((torch.from_numpy(tcls)[:, None], tbox), 1)
        else:
            targets = torch.zeros((0, 5))

        for i, det in enumerate(preds):
            seen += 1
            if len(det) == 0:
                if nl: stats.append((torch.zeros(0, 10, dtype=torch.bool), torch.Tensor(), torch.Tensor(), torch.from_numpy(tcls)))
                continue

            if nl:
                correct = torch.zeros(len(det), 10, dtype=torch.bool)
                iou = box_iou(targets[:, 1:], det[:, :4])
                for j, v in enumerate(iouv):
                    x = torch.where((iou >= v) & (targets[:, 0:1] == det[:, 5]))
                    if x[0].shape[0]:
                        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()
                        matches = matches[matches[:, 2].argsort()[::-1]]
                        matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                        matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
                        correct[matches[:, 1].astype(int), j] = True

            stats.append((correct.cpu(), det[:, 4].cpu(), det[:, 5].cpu(), torch.from_numpy(tcls).cpu()))

    # Calculate final mAP
    stats = [torch.cat(x, 0).numpy() for x in zip(*stats)]
    if len(stats) and stats[0].any():
        tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, names=data['names'])
        print(f"\n✅ SUCCESS! mAP50: {ap[:, 0].mean():.4f}")
        return ap[:, 0].mean()
    else:
        print("❌ Still failing. The INT8 model is likely outputting garbage values.")
        return 0

evaluate_onnx_accuracy("/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/rice_nano_final_1mb.onnx", "/content/output_folder/content/yolov5/data.yaml")

🚀 Running Robust Evaluation on 15 images...


100%|██████████| 15/15 [00:10<00:00,  1.38it/s]


AttributeError: 'list' object has no attribute 'items'

In [47]:
!pip install onnxruntime-gpu onnx

In [55]:
import yaml
import torch
import numpy as np
import onnxruntime as ort
from utils.general import check_yaml, non_max_suppression, xywh2xyxy, box_iou
from utils.metrics import ap_per_class
from utils.dataloaders import create_dataloader

def evaluate_onnx_fixed(onnx_path, data_yaml, imgsz=1024):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    with open(data_yaml, 'r') as f:
        data = yaml.safe_load(f)

    # Load ONNX
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if device.type == 'cuda' else ['CPUExecutionProvider']
    session = ort.InferenceSession(onnx_path, providers=providers)
    input_name = session.get_inputs()[0].name

    # FIX: rect=False is mandatory for fixed-size ONNX models
    test_path = '/content/output_folder/content/yolov5/test/images'
    dataloader = create_dataloader(test_path, imgsz, batch_size=1, stride=32, pad=0.5, rect=False)[0]

    stats = []
    for im, targets, paths, shapes in dataloader:
        im = im.to(device).float() / 255
        if len(im.shape) == 3: im = im[None]

        # Inference
        outputs = session.run(None, {input_name: im.cpu().numpy()})
        out = torch.from_numpy(outputs[0]).to(device)

        # NMS
        preds = non_max_suppression(out, 0.001, 0.6)

        for i, det in enumerate(preds):
            targ = targets[targets[:, 0] == i, 1:]
            nl = len(targ)
            tcls = targ[:, 0].tolist() if nl else []
            if len(det) == 0:
                if nl: stats.append((torch.zeros(0, 10, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            correct = torch.zeros(det.shape[0], 10, dtype=torch.bool, device=device)
            if nl:
                # Rescale boxes from 1024 to original image shapes
                tbox = xywh2xyxy(targ[:, 1:5]) * torch.tensor([imgsz, imgsz, imgsz, imgsz], device=device)
                iou = box_iou(tbox, det[:, :4])
                matches = (iou >= 0.5) & (targ[:, 0:1] == det[:, 5])
                if matches.any():
                    for j in range(len(det)):
                        if matches[:, j].any(): correct[j, 0] = True

            stats.append((correct.cpu(), det[:, 4].cpu(), det[:, 5].cpu(), tcls))

    # Calculate mAP
    stats = [np.concatenate(x, 0) for x in zip(*stats)]
    tp, fp, p, r, f1, ap, unique_classes = ap_per_class(*stats, names=data['names'])
    print(f"mAP50 for {onnx_path}: {ap[:, 0].mean():.4f}")


# Convert the list of names from your YAML into a dictionary for the metrics function
with open('/content/output_folder/content/yolov5/data.yaml', 'r') as f:
    import yaml
    data_dict = yaml.safe_load(f)

# Create the dictionary: {0: 'broken-chalky', 1: 'broken-clear', ...}
names_dict = dict(enumerate(data_dict['names']))

# Execute with the corrected names format
evaluate_onnx_fixed(
    onnx_path='/content/output_folder/content/yolov5/runs/train/rice_nano_QAT_final/weights/rice_nano_final_1mb.onnx',
    data_yaml='/content/output_folder/content/yolov5/data.yaml',
    imgsz=1024
)

Scanning /content/output_folder/content/yolov5/test/labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'items'

In [49]:
import yaml
with open('/content/output_folder/content/yolov5/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
print(f"Number of classes: {data['nc']}")
print(f"Class names: {data['names']}")

Number of classes: 8
Class names: ['broken-chalky', 'broken-clear', 'foreign-object', 'plastic', 'sound', 'stone', 'unsound', 'whole-chalky']
